In [ ]:
import nasim
from nasim.agents.ql_replay_agent import *

class GymObsCompatWrapper:
    def __init__(self, env):
        self.env = env
    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        if isinstance(obs, tuple):
            return obs[0]  
        return obs
    def step(self, action):
        res = self.env.step(action)
        if isinstance(res[0], tuple):
            obs = res[0][0]  
            rest = res[1:]
            return (obs, *rest)
        else:
            return res
    def __getattr__(self, x):
        return getattr(self.env, x)

yaml_path = "medium_with_honeypot.yaml" 
env = nasim.load(yaml_path, fully_obs=True, flat_actions=True, flat_obs=True, render_mode=None)
env = GymObsCompatWrapper(env)
ql_agent = TabularQLearningAgent(env,
                 seed=100,
                 lr=0.001,
                 training_steps=2000,
                 batch_size=64,
                 replay_size=10000,
                 final_epsilon=0.01,
                 exploration_steps=10000,
                 gamma=0.99,
                 verbose=False)


ql_agent.train()

episode_return, steps, done = ql_agent.run_eval_episode(env,
                         render=False,
                         eval_epsilon=0.01,
                         render_mode="readable")
print(episode_return)
print(steps)
print(done)